In [2]:
# Adapted from https://github.com/HobbitLong/SupContrast/blob/master/losses.py

import torch
import torch.nn as nn


class SupConLoss(nn.Module):
    def __init__(self, temperature=0.07, contrast_mode='all'):
        super(SupConLoss, self).__init__()
        self.temperature = temperature
        self.contrast_mode = contrast_mode

    def forward(self, features, labels=None, mask=None):
        """Compute loss for model. If both `labels` and `mask` are None,
        it degenerates to SimCLR unsupervised loss:
        https://arxiv.org/pdf/2002.05709.pdf
        Args:
            features: hidden vector of shape [bsz, n_views, ...].
            labels: ground truth of shape [bsz].
            mask: contrastive mask of shape [bsz, bsz], mask_{i,j}=1 if sample j
                has the same class as sample i. Can be asymmetric.
        Returns:
            A loss scalar.
        """
        device = (torch.device('cuda')
                  if features.is_cuda
                  else torch.device('mps') if features.is_mps else torch.device('cpu'))

        if len(features.shape) < 3:
            raise ValueError('`features` needs to be [bsz, n_views, ...],'
                             'at least 3 dimensions are required')
        if len(features.shape) > 3:
            features = features.view(features.shape[0], features.shape[1], -1)

        batch_size = features.shape[0]
        if labels is not None and mask is not None:
            raise ValueError('Cannot define both `labels` and `mask`')
        elif labels is None and mask is None:
            mask = torch.eye(batch_size, dtype=torch.float32).to(device)
        elif labels is not None:
            labels = labels.contiguous().view(-1, 1)
            if labels.shape[0] != batch_size:
                raise ValueError('Num of labels does not match num of features')
            mask = torch.eq(labels, labels.T).float().to(device)
        else:
            mask = mask.float().to(device)

        contrast_count = features.shape[1]
        contrast_feature = torch.cat(torch.unbind(features, dim=1), dim=0)
        if self.contrast_mode == 'one':
            anchor_feature = features[:, 0]
            anchor_count = 1
        elif self.contrast_mode == 'all':
            anchor_feature = contrast_feature
            anchor_count = contrast_count
        else:
            raise ValueError('Unknown mode: {}'.format(self.contrast_mode))

        # compute logits
        anchor_dot_contrast = torch.div(
            torch.matmul(anchor_feature, contrast_feature.T),
            self.temperature)
        # for numerical stability
        logits_max, _ = torch.max(anchor_dot_contrast, dim=1, keepdim=True)
        logits = anchor_dot_contrast - logits_max.detach()

        # tile mask
        mask = mask.repeat(anchor_count, contrast_count)
        # mask-out self-contrast cases
        logits_mask = torch.scatter(
            torch.ones_like(mask),
            1,
            torch.arange(batch_size * anchor_count).view(-1, 1).to(device),
            0
        )
        mask = mask * logits_mask
        # compute log_prob
        exp_logits = torch.exp(logits) * logits_mask
        log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True)).nan_to_num(0, 0, 0)
        # compute mean of log-likelihood over positive
        mean_log_prob_pos = (mask * log_prob).sum(1) / mask.sum(1)
        # loss
        loss = - mean_log_prob_pos.nan_to_num(0, 0, 0)
        loss = loss.view(anchor_count, batch_size).mean()

        return loss

In [1]:
import torch
from torch import nn
from utils.contrastive_loss import SupConLoss

class LabelImageAttention(nn.Module):
    def __init__(self, dim_in, n_head, dropout=0.1, num_layers=4, temperature=1):
        super().__init__()
        self.attn = nn.Transformer(dim_in, batch_first=True, nhead=n_head, dropout=dropout, num_decoder_layers=num_layers, num_encoder_layers=num_layers)
        self.con_loss = SupConLoss(temperature=temperature)

    def forward(self, texts, images, label_inds=None):
        # transformer: (N, S, E), (N, T, E) -> (N, T, E)
        # texts: (L,D) , images: (N,D,H,W), label_inds: (N, L)
        texts = texts.repeat(images.shape[0], 1, 1)
        # view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.
        images = images.flatten(start_dim=2).permute(0, 2, 1)
        mask = None
        if label_inds is not None:
            mask = (1 - label_inds).bool()
        # Output: (N, L, D)
        # Texts: NxLxD (decode)
        # Mask irrelevant labels with tgt_key_padding_mask, set masked positions to True
        # Images: Nx(HxW)xD
        out = self.attn(images, texts, tgt_key_padding_mask=mask)
        return out
    
    def loss(self, results, label_inds):
        # results: (N, L, D), labels: (N, L)
        classes = torch.nonzero(label_inds)[:,1] # (Np,)
        prototypes = results[label_inds.bool()] # (Np, D)
        return self.con_loss(prototypes.unsqueeze(1), classes)


class LabelImagePrototypeModel(nn.Module):
    def __init__(self, encoder, n_head, dim_in=512, dropout=0.1, num_layers=4, temperature=1):
        super().__init__()
        self.encoder = encoder
        self.attention = LabelImageAttention(dim_in, n_head, dropout, num_layers, temperature)

    def freeze_encoder(self):
        for param in self.encoder.parameters():
            param.requires_grad = False

    def unfreeze_encoder(self):
        self.encoder.img_model.proj.requires_grad = True
        self.encoder.text_model.proj.requires_grad = True
    
    def forward(self, class_labels, images, label_inds):
        text_embedding, image_emedding = self.encoder(class_labels, images, False)
        prototypes = self.attention(text_embedding, image_emedding, label_inds)
        return text_embedding, image_emedding, prototypes
    
    def attention_loss(self, prototypes, label_inds):
        return self.attention.loss(prototypes, label_inds)


In [2]:
import torch
import copy
from utils.metrics import AverageMeter, calculate_auc, multilabel_accuracy
from utils.prototype import class_prototype_mean

class Trainer:
    def __init__(self, model, class_labels, device='cpu'):
        self.model = model
        self.device = device
        self.class_labels = class_labels
    
    def run_train(self, epochs, dataloader, val_dataloader, lr=1e-4, full_training=False):
        model = self.model.to(self.device)
        best_epoch = None
        best_acc = None
        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        if full_training:
            model.unfreeze_encoder()
        else:
            model.freeze_encoder()
        
        for epoch in range(epochs):
            model.train()
            loss_meter = AverageMeter()
            for i, (images, class_inds) in enumerate(dataloader):
                images, class_inds = images.to(self.device), class_inds.to(self.device)
                optimizer.zero_grad()

                text_embeddings, _, prototypes = model(self.class_labels, images, class_inds)
                loss = model.attention_loss(prototypes, class_inds)
                if full_training:
                    class_prototypes = class_prototype_mean(prototypes)
                    loss += 0.5 * self.encoder.loss(text_embeddings, class_prototypes, class_inds)
               
                loss.backward()
                optimizer.step()

                loss_meter.update(loss.item(), len(class_inds))
                print(f"Batch {i+1}: loss {loss_meter.average()}")
            
            print(f"Epoch {epoch+1}: Training loss {loss_meter.average()}")
            val_acc, val_auc, val_loss = self.run_eval(model, val_dataloader)
            print(f"Epoch {epoch+1}: Validation loss {val_loss} | Accuracy {val_acc} | AUC {val_auc}")
            
            if best_acc is None or val_acc > best_acc:
                best_acc = val_acc
                self.best_model = copy.deepcopy(model)
                best_epoch = epoch
        print('Best epoch: ', best_epoch+1)

    def run_eval(self, model, dataloader, full_training=False):
        model.eval()
        model = model.to(self.device)
        
        loss_meter = AverageMeter()
        auc_meter = AverageMeter()
        acc_meter = AverageMeter()
        with torch.no_grad():
            for images, class_inds in dataloader:
                images, class_inds = images.to(self.device), class_inds.to(self.device)
                text_embeddings, _, prototypes = model(self.class_labels, images, class_inds)
                class_prototypes = class_prototype_mean(prototypes)
                logits_per_text, logits_per_image = model.encoder.compute_logits(text_embeddings, class_prototypes)

                loss = model.attention_loss(prototypes, class_inds).item()
                if full_training:
                    loss += model.encoder.contrastive_logit_loss(logits_per_text, logits_per_image, class_inds).item()
        
                loss_meter.update(loss, len(class_inds))

                auc = calculate_auc(logits_per_image, class_inds)
                auc_meter.update(auc, len(class_inds))
                
                acc = multilabel_accuracy(logits_per_image, class_inds)
                acc_meter.update(acc, len(class_inds))
        return acc_meter.average(), auc_meter.average(), loss_meter.average()

In [3]:
import torch
from torch.utils.data import DataLoader
import pandas as pd

from models.embedding.model import ImageTextEmbedding, TextEncoder, ImageEncoder
from utils.labels import VINDR_CXR_LABELS, VINDR_SPLIT
from models.backbone.datasets import MEAN_STDS
from utils.device import get_device
from utils.data import get_query_and_support_ids
from models.embedding.dataset import Dataset

img_info = pd.read_pickle('data/vindr_cxr_split_labels.pkl')
query_image_ids, support_image_ids = get_query_and_support_ids(img_info, 'data/vindr_train_query_set.pkl')

IMG_PATH = 'datasets/vindr-cxr-png'
batch_size = 10*14
query_dataset = Dataset(IMG_PATH, img_info, query_image_ids, VINDR_CXR_LABELS, VINDR_SPLIT['train'], mean_std=MEAN_STDS['chestmnist'])
query_loader = DataLoader(dataset=query_dataset, batch_size=batch_size, shuffle=True)
support_dataset = Dataset(IMG_PATH, img_info, support_image_ids, VINDR_CXR_LABELS, VINDR_SPLIT['train'], mean_std=MEAN_STDS['chestmnist'])
support_loader = DataLoader(dataset=support_dataset, batch_size=batch_size, shuffle=True)

PROJ_SIZE = 512
# device = 'cpu'
device =  get_device()

encoder = torch.load('imgtext_model_trained1-newlib.pth')
encoder.text_model.device = device
model = LabelImagePrototypeModel(encoder, 8, PROJ_SIZE)
mtrainer = Trainer(model, support_dataset.class_labels(), device)

In [4]:
mtrainer.run_train(5, support_loader, query_loader, lr=1e-4, full_training=False)

/var/folders/ny/2ylc61455nv8t8z2px9ppsqw0000gn/T/ipykernel_92060/1616687668.py:29: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:11.)
  classes = torch.nonzero(label_inds)[:,1] # (Np,)


NotImplementedError: The operator 'aten::nan_to_num.out' is not currently implemented for the MPS device. If you want this op to be added in priority during the prototype phase of this feature, please comment on https://github.com/pytorch/pytorch/issues/77764. As a temporary fix, you can set the environment variable `PYTORCH_ENABLE_MPS_FALLBACK=1` to use the CPU as a fallback for this op. WARNING: this will be slower than running natively on MPS.

In [93]:
mtrainer.run_eval(mtrainer.model, query_loader)

/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/transformers/attention.cpp:152.)
  return torch._native_multi_head_attention(
/opt/homebrew/anaconda3/envs/cs6240/lib/python3.9/site-packages/torch/nn/modules/activation.py:1144: UserWarning: Converting mask without torch.bool dtype to bool; this will nega

(41.07142857142857, 0.5212234249734956, 77.43980865478515)

In [ ]:
mtrainer.run_eval(mtrainer.best_model, query_loader)